# Shiuli Subhra Ghosh(MDS202035), Suman Roy(MDS202041)

# Parity Checker using Deep Neural Network 
## Date: 19.11.2021

## Data Set Generator train (n = 2000), test (n = 50)

Algorithm used for generate the randomized data set:
- First we have chosen 2050/5050 (including test data points) random integers in between the range 0 and 100000. 
- Then we have obtained the binary representation for each numbers and converted it in list of strings.
- Then while converting the list of string to list of list, we have randomly shuffeled each data point. 
- Finally we have converted in numpy 2-D array and store it in a Data Frame
- We also saved the simulated data in **.csv** format ('gen2000.csv' and 'gen5000.csv') and these 2 data sets were used in for this assignment.

In [153]:
import random
import pandas as pd
import numpy as np

random.seed(42)
 
n = 64
t = [random.randint(0, 100000) for p in range(0, 2050)]
l = [bin(x)[2:].rjust(n, '0') for x in t]

def Convert(string):
    list1=[]
    list1[:0]=string
    return list1

for i in range(len(l)):
    ls = Convert(l[i])
    random.shuffle(ls)
    l[i] = np.array(list(map(int,ls)))
    
l = np.array(l)    
    
df = pd.DataFrame(l)   

y = [sum(i)%2 for i in l]  # 0 means Even parity 1 means odd parity
df['Y'] = y

df.to_csv('gen2000.csv')

## Data Set Generator train (n = 5000) test (n = 50)

In [154]:
random.seed(42)
n = 64
t = [random.randint(0, 100000) for p in range(0, 5050)]
l = [bin(x)[2:].rjust(n, '0') for x in t]

def Convert(string):
    list1=[]
    list1[:0]=string
    return list1

for i in range(len(l)):
    ls = Convert(l[i])
    random.shuffle(ls)
    l[i] = np.array(list(map(int,ls)))
    
l = np.array(l)    
    
df = pd.DataFrame(l)   

y = [sum(i)%2 for i in l]  # 0 means Even parity 1 means odd parity
df['Y'] = y

df.to_csv('gen5000.csv')

## Parity Checker using DNN

### Importing Packages

In [135]:
import torch
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import Adam
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [136]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Data Set Definition

In [137]:
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, file_name):
        # load the csv file as a dataframe
        df = read_csv(file_name, header=0, index_col = 0)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        
        # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=50):
        # determine sizes
        test_size = n_test
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

### Model Definition (Two hidden layers)

Here, we have used 2 hidden layers in this case. In the first hidden layers we have taken 100 neurons and in the 2nd hidden layer we have taken 10 neurons. 

In [152]:
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 100)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(100, 10)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(10, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

### Data Set preparation

In [139]:
def prepare_data(file_name):
    # load the dataset
    dataset = CSVDataset(file_name)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=64, shuffle=True)
    test_dl = DataLoader(test, batch_size=64, shuffle=False)
    return train_dl, test_dl

### Model training

We have used Adam optimizer and BCELoss as we have binary class classification problem.

In [140]:
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = Adam(model.parameters())
    # enumerate epochs
    for epoch in range(10):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
        print(loss)

### Model Evaluation

In [122]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

### Model Prediction

In [141]:
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat   

# Driver Code

#### First running the model on 'gen2000.csv'

In [172]:
file_name = r'gen2000.csv'
train, test = prepare_data(file_name) 
print(len(train.dataset), len(test.dataset))

2000 50


In [175]:
model = MLP(64)
train_model(train, model)

tensor(0.6898, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6945, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7320, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6294, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7141, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6796, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6387, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6009, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6292, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6474, grad_fn=<BinaryCrossEntropyBackward0>)


In [176]:
accuracy = evaluate_model(test, model)*100
accuracy_1 = evaluate_model(train , model)*100
print('test_Accuracy: %.3f' % accuracy)
print('train_Accuracy: %.3f' % accuracy_1)

test_Accuracy: 54.000
train_Accuracy: 71.700


In [177]:
Accuracy_matrix = pd.DataFrame(index = ('test_accuracy','train_accuracy'))

In [178]:
accu = [accuracy,accuracy_1]
Accuracy_matrix['gen2000'] = accu
Accuracy_matrix

,gen2000
test_accuracy,54.0
train_accuracy,71.7


#### Now, training on 'gen5000.csv'

In [179]:
file_name = r'gen5000.csv'
train, test = prepare_data(file_name) 
print(len(train.dataset), len(test.dataset))

5000 50


In [182]:
model = MLP(64)
train_model(train, model)

tensor(0.7038, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6671, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6774, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6450, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6666, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6491, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6036, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6320, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6588, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.5466, grad_fn=<BinaryCrossEntropyBackward0>)


In [183]:
accuracy = evaluate_model(test, model)*100
accuracy_1 = evaluate_model(train , model)*100
print('test_Accuracy: %.3f' % accuracy)
print('train_Accuracy: %.3f' % accuracy_1)

test_Accuracy: 58.000
train_Accuracy: 70.400


# Accumulating train and test accuracies

In [184]:
accu = [accuracy,accuracy_1]
Accuracy_matrix['gen5000'] = accu
Accuracy_matrix

,gen2000,gen5000
test_accuracy,54.0,58.0
train_accuracy,71.7,70.4


We can observe that for both the Data sets, the training accuracy is nearly 70% and the test accuracy is in between 45% to 55%. It is because, for 64 bit binary numbers, there are $2^{64}$ numbers of elements are present in space and we are randomly selecting 2000 for the first and 5000 for the second case. So, it is not possible to obtain greater accuracy in case of test set as the training is totally randomised in nature. 
- Taking 2000 data points for training is equivalent to take 5000 data points for training.  

# Implementing Same Architecture as the Manual Implementation (One hidden layer)

Here, I have tried to check if we train the NN with same architecture what is the loss and the accuracy of the model.

In [212]:
class MLP_1(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP_1, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 96)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(96, 1)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        return X

In [213]:
file_name = r'gen2000.csv'
train, test = prepare_data(file_name) 
print(len(train.dataset), len(test.dataset))

2000 50


In [214]:
model = MLP_1(64)
train_model(train, model)

tensor(0.7238, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6546, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6850, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7461, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7189, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6669, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7098, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6142, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6657, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6671, grad_fn=<BinaryCrossEntropyBackward0>)


In [215]:
accuracy = evaluate_model(test, model)*100
accuracy_1 = evaluate_model(train , model)*100
print('test_Accuracy: %.3f' % accuracy)
print('train_Accuracy: %.3f' % accuracy_1)

test_Accuracy: 52.000
train_Accuracy: 65.150


After training it is evident that the learning is not perfect even if we are using the same architecture as we used for the manual implementation. The accuracy is almost equvant to the previous network with 2 hidden layers. So, from here, we can also verify that one hidden layer is sufficient to implement any logic circuits. 
- Finally our manually designed network is definitely performing better than the Deep network.